In [1]:
import numpy as np
import pandas as pd
import glob
import os
os.getcwd()
os.chdir('/project/time_series')
from load_data_custom import load_data_using_multi_process
from make_roc_curve import make_roc_curve
os.chdir('/project/time_series/cchc0427')
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from model import model_lgbm
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, precision_score, recall_score
import lazy_loader as lazy
samplerate = lazy.load("samplerate")
resampy = lazy.load("resampy")
import copy
import librosa
import time
import csv
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
fan_path = '/data/time_series/fan/0'
id_lst = ['/id_00', '/id_02', '/id_04', '/id_06']
file_path = []        

In [3]:
for i, test_id in enumerate(id_lst):
    id_lst1 = copy.deepcopy(id_lst)
    id_lst1.remove(test_id)
    test_file = glob.glob(fan_path + test_id +'/*/*')
    train_file = []
    
    obj_name = test_file[0].split('/')[3:6]
    print(f'{test_id} 시작')
    print("_".join(obj_name))
    
    for train_id in id_lst1:
        file = glob.glob(fan_path + train_id + '/*/*')
        train_file += file

/id_00 시작
fan_0_id_00
/id_02 시작
fan_0_id_02
/id_04 시작
fan_0_id_04
/id_06 시작
fan_0_id_06


In [4]:
target_lst = []
for i in range(1, 16001):
    if ( i < 60 ) or ( i > 13000):
        continue
    if 16000 % i == 0 :
        target_lst.append(int(i))
target_lst.append(10000)
target_lst.append(13000)
sampling_lst = ['abs_max(our_custom)',
                'kaiser_best',
                'kaiser_fast',
                'fft',
                'scipy',
                'polyphase',
                'linear',
                'zero_order_hold',
                'sinc_best',
                'sinc_medium' ,
                'sinc_fastest',
                'soxr_vhq',
                'soxr_hq',
                'soxr_mq',
                'soxr_lq',
                'soxr_qq']

In [6]:
obj_name = test_file[0].split('/')[3:6]
name = "_".join(obj_name)
name = str(name)
name = str(name)

In [7]:
id_lst

['/id_00', '/id_02', '/id_04', '/id_06']

In [8]:
print(f'행개수{len(sampling_lst) * len(target_lst) * len(id_lst)}')

행개수1344


In [9]:
print(f'행개수{len(sampling_lst) * len(target_lst) * 12}')

행개수4032


In [20]:
len(train_file)

4132

In [18]:
len(test_file)

1418

In [10]:
# 여기 돌려요
result = []
f = open('cv_id.csv','w', newline = '')
wr = csv.writer(f)
wr.writerow(['Model','sampling_way','target_sr','acc','recall','precision','f1','auc_score'])

for i, test_id in enumerate(id_lst):
    id_lst1 = copy.deepcopy(id_lst)
    id_lst1.remove(test_id)
    test_file = glob.glob(fan_path + test_id +'/*/*')
    train_file = []
    
    obj_name = test_file[0].split('/')[3:6]
    name = str('_'.join(obj_name))
    
    for train_id in id_lst1:
        file = glob.glob(fan_path + train_id + '/*/*')
        train_file += file
    break    
    for sampling_way in sampling_lst:
        start = time.time()
        for target_sr in target_lst:

            train_result = load_data_using_multi_process(train_file, sr = 16000, target_sr = target_sr, sampling_way = sampling_way) # 데이터 target_sr로 부르기
            train_result_df = pd.DataFrame(train_result)

            test_result = load_data_using_multi_process(test_file, sr = 16000, target_sr = target_sr, sampling_way = sampling_way)
            test_result_df = pd.DataFrame(test_result)
            
            train_data = train_result_df.iloc[:, :-1]
            train_target = train_result_df.iloc[:, -1]

            test_data = test_result_df.iloc[:, :-1]
            test_target = test_result_df.iloc[:, -1]
            
            model = model_lgbm(train_result_df) # using (model.py)

            model.fit(train_data, train_target)

            y_pred = model.predict(test_data)
            y_pred_prob = model.predict_proba(test_data)

            acc = accuracy_score(test_target, y_pred) 
            recall = recall_score(test_target, y_pred)
            precision = precision_score(test_target, y_pred)
            f1 = f1_score(test_target, y_pred)    
            auc_score = roc_auc_score(test_target, y_pred_prob[ :, 1 ])

            result += [ str(name), str(sampling_way), target_sr, round(acc, 3), round(recall, 3), round(precision, 3), round(f1, 3), round(auc_score, 3) ]
            print(f'{result}\n')
            wr.writerow(result)
            result = []
            
f.close()
print('파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료\n\n' * 3)

파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료

파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료

파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료파일 저장 완료


